This program reads a tab separated (tsv) file containing the author name, affiliation, city and country, and generate the coordinates corresponding to their institude.

One line of the file looks like this:     
`Adam	Y.	Université de Liège	Liège	Belgium`      
The city column was added to make easier the localisation.

For each year, two output files are created:
1. a file storing only the coordinates, that can be used directly with leaflet (see [leaflet](./leaflet) folder) 
2. a file that is the same as the input file but with 2 extra columns for the coordinates.

If the coordinates cannot be found for some reasons, a fill value (e.g. -999) is used, and the coordinates will have to be found "manually", i.e., using other resources.

In [1]:
import os
import glob
import logging
from collections import Counter
from liegecolloquium import Participant
from importlib import reload

# Configuration

In [2]:
datadir = "../data/tsv/"
outputdir = "../data/processed20180528/"
coordfileall = os.path.join(outputdir, "coordsAffil.dat")
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [10]:
filelist = sorted(glob.glob(os.path.join(datadir, "ParticipantList-*.tsv")))
logging.info("Working on {0} file(s)".format(len(filelist)))

INFO:root:Working on 39 file(s)


In [11]:
participantlist = []
locationlist = []
startfile = 29

logger.info("Starting with file {0}".format(filelist[startfile]))

if not(os.path.exists(outputdir)):
    os.makedirs(outputdir)
    
# Write full file witch coordinates and institute
with open(coordfileall, 'w') as cfall:
    cfall.write("var coords = [\n")

for datafiles in filelist[startfile:]:
    np = 0
    logging.info("Working on file {0}".format(datafiles))
    outputbasename = os.path.basename(datafiles).replace(".tsv", ".dat")
    coordbasename = os.path.basename(datafiles).replace(".tsv", ".coord")
    outputfile = os.path.join(outputdir, outputbasename)
    coordfile = os.path.join(outputdir, coordbasename)
    logging.info("Writing info in file {0}".format(outputfile))
    logging.info("Writing coordinates in file {0}".format(outputfile))
    logging.info("----------------")
    
    with open(coordfile, 'w') as cf:
        cf.write("var coords = [\n")

    with open(datafiles, 'r') as f:
        for line in f:
            
            if line == '\n':
                logger.debug("Empty line")
            else:
                np += 1
                l = line.rstrip().split('\t')
                participant = Participant(l[0], l[1], l[2], l[3], l[4])
                logger.debug("Working with {0}".format(participant))

                # Modify country name via dictionary
                participant.replace_country()

                # Find location ; check if in the list before using geopy
                loc1 = ", ".join((participant.affiliation, 
                                  participant.city, 
                                  participant.country))
                
                # logger.debug(loc1)

                if loc1 in locationlist:
                    logger.debug("Already in the list")
                    ind = locationlist.index(loc1)
                    logger.debug("Position in the list: {0}".format(ind))
                    participant.lat = participantlist[ind].lat
                    participant.lon = participantlist[ind].lon
                    participant.countryname = participantlist[ind].countryname
                    locationlist.append(locationlist.index(loc1))
                else:
                    logger.debug("Not in the list, use geolocator to get coordinates")
                    locationstring, location = participant.get_location()
                    locationlist.append(locationstring)

                # Write info to a text file
                participant.write_to(outputfile)

                # Write coordinates (only) into another file
                participant.write_coords_to(coordfile)
                
                # Write coords + affil in a global file
                with open(coordfileall, 'a') as cfall:
                    cfall.write("[{0}, {1}, {2}],\n".format(participant.lat, 
                                                            participant.lon,
                                                            participant.affiliation))

                # Append to list
                participantlist.append(participant)
                
        with open(coordfile, 'a') as cf:
            cf.write("]")
               
            
with open(coordfileall, 'a') as cfall:
    cfall.write("]")

logger.info("{} participants".format(np))
logger.info("\n Job finished")

INFO:root:Starting with file ../data/tsv/ParticipantList-2008.tsv
INFO:root:Working on file ../data/tsv/ParticipantList-2008.tsv
INFO:root:Writing info in file ../data/processed20180528/ParticipantList-2008.dat
INFO:root:Writing coordinates in file ../data/processed20180528/ParticipantList-2008.dat
INFO:root:----------------
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.os

Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim

Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim

Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim

Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim

...

Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim

Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim

Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim

Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim

Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim object at 0x7eff5043fa58>
Problem with geocoder <geopy.geocoders.osm.Nominatim